In [2]:
import numpy as np
import pandas as pd
import cv2

In [8]:
IN="../Data/Images/Aligned/"

In [9]:
latlon = cv2.imread(IN+"LatLon_aligned.png")
N  = cv2.imread(IN+"N_aligned.png")
P  = cv2.imread(IN+"P_aligned.png")
K  = cv2.imread(IN+"K_aligned.png")
PH = cv2.imread(IN+"pH_aligned.png")

h, w, _ = latlon.shape

In [10]:
h, w, _ = latlon.shape

In [13]:
latitudes  = np.linspace(20, 13, h)
longitudes = np.linspace(77, 84, w)

In [14]:
gray = cv2.cvtColor(latlon, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(gray, 245, 255, cv2.THRESH_BINARY_INV)
ap_mask = mask > 0

In [15]:
cv2.imshow("AP Mask", (ap_mask*255).astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
def nitrogen_value(pixel):
    hsv = cv2.cvtColor(np.uint8([[pixel]]), cv2.COLOR_BGR2HSV)[0][0]
    h,s,v = hsv
    if s < 80: return np.nan
    if (h < 10 or h > 170): return 140.0
    if 20 < h < 35: return 420.0
    return np.nan

In [17]:
def phosphorus_value(pixel):
    hsv = cv2.cvtColor(np.uint8([[pixel]]), cv2.COLOR_BGR2HSV)[0][0]
    h,s,v = hsv
    if s < 80: return np.nan
    if h < 15 or h > 165: return 5.0
    if 15 <= h <= 40: return 17.5
    if 40 < h <= 100: return 40.0
    return np.nan

In [18]:
def potassium_value(pixel):
    hsv = cv2.cvtColor(np.uint8([[pixel]]), cv2.COLOR_BGR2HSV)[0][0]
    h,s,v = hsv
    if s < 80: return np.nan
    if h < 10 or h > 170: return 60.0
    if 20 < h < 35: return 200.0
    if 35 < h < 100: return 450.0
    return np.nan


In [19]:
def ph_value(pixel):
    hsv = cv2.cvtColor(np.uint8([[pixel]]), cv2.COLOR_BGR2HSV)[0][0]
    h,s,v = hsv
    if s < 60: return np.nan
    if (h < 10 or h > 170) and v < 150: return 4.0
    if s < 30 and 100 < v < 200: return 5.0
    if 10 < h < 20: return 6.0
    if 35 < h < 85: return 7.0
    if 140 < h < 165: return 8.0
    if 15 < h < 25 and v < 200: return 9.0
    if 120 < h < 140: return 10.0
    return np.nan

In [20]:
rows = []

for i in range(h):
    for j in range(w):

        if not ap_mask[i,j]:
            continue

        n = nitrogen_value(N[i,j])
        if np.isnan(n):
            continue   # skip rivers, no-data

        rows.append([
            latitudes[i],
            longitudes[j],
            n,
            phosphorus_value(P[i,j]),
            potassium_value(K[i,j]),
            ph_value(PH[i,j])
        ])

In [49]:
df = pd.DataFrame(rows, columns=[
    "latitude","longitude",
    "nitrogen_kg_ha",
    "phosphorus_kg_ha",
    "potassium_kg_ha",
    "pH"
])

df = df.round(2)
df.to_csv(r'..\Data\Csv_Files\AndhraSoil_Data.csv', index=False)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101242 entries, 0 to 101241
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   latitude          101242 non-null  float64
 1   longitude         101242 non-null  float64
 2   nitrogen_kg_ha    101242 non-null  float64
 3   phosphorus_kg_ha  97259 non-null   float64
 4   potassium_kg_ha   93188 non-null   float64
 5   pH                85066 non-null   float64
dtypes: float64(6)
memory usage: 4.6 MB
